Part 1 70%
Write a jupyter notebook that analyses the differences between the sexes by age in Ireland.

* Weighted mean age (by sex)
* The difference between the sexes by age

This part does not need to look at the regions.



Part 2 20%

* make a variable that stores an age (say 35).

* Write that code that would group the people within 5 years of that age together, into one age group 

* Calculate the population difference between the sexes in that age group.

Part 3 10%


* Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group

In [162]:
import pandas as pd
import numpy as np

In [163]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.head()

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
0,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,IE0,Ireland,Number,5149139
1,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-1492-13a3-e055-000000000001,Carlow County Council,Number,61968
2,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-1433-13a3-e055-000000000001,Dublin City Council,Number,592713
3,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-149f-13a3-e055-000000000001,Dún Laoghaire Rathdown County Council,Number,233860
4,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-14a0-13a3-e055-000000000001,Fingal County Council,Number,330506


In [164]:
# Ensure columns are correctly named, e.g., 'Sex', 'Age', 'Population'
# Adjust column names if needed
df.columns = df.columns.str.strip()


In [165]:
headers = df.columns.tolist()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [166]:
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','C02076V03371','C03789V04537','UNIT']
df.drop(columns=drop_col_list, inplace=True)
df = df[df["Single Year of Age"] != "All ages"]
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\\D', '', regex=True)


df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE'] = df['VALUE'].round(0).astype('int64')



# Round all numeric columns to nearest integer and convert to int
df[df.select_dtypes(include='number').columns] = df.select_dtypes(include='number').round(0).astype('int64')
#print (df.head(3))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9696 entries, 32 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      9696 non-null   object
 1   Single Year of Age       9696 non-null   int64 
 2   Administrative Counties  9696 non-null   object
 3   VALUE                    9696 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 378.8+ KB


In [167]:
df_anal = pd.pivot_table(df, 'VALUE',"Single Year of Age","Sex")

# Ensure numeric values are integers
df_anal[df_anal.select_dtypes(include='number').columns] = df_anal.select_dtypes(include='number').round(0).astype('int64')


print (df_anal.head(5))
# write out the entire file to local machine
df_anal.to_csv("population_for_analysis.csv")

Sex                 Both sexes  Female  Male
Single Year of Age                          
0                         3612    1762  1851
1                         3526    1722  1805
2                         3701    1811  1890
3                         3780    1843  1938
4                         3844    1864  1980


In [168]:
headers = list(df_anal.columns)
Both_sexes = headers[0]
female = headers[1]
male = headers[2]

print("Columns in pivoted dataframe:", headers)

Columns in pivoted dataframe: ['Both sexes', 'Female', 'Male']


In [169]:
# Mean age calculations (by sex)
mean_males = (df_anal.index * df_anal['Male']).sum() / df_anal['Male'].sum()
mean_females = (df_anal.index * df_anal['Female']).sum() / df_anal['Female'].sum()
mean_both = (df_anal.index * df_anal['Both sexes']).sum() / df_anal['Both sexes'].sum()

# Convert to integers (rounded) for display
mean_males_int = int(round(mean_males))
mean_females_int = int(round(mean_females))
mean_both_int = int(round(mean_both))

print("mean-males:", mean_males_int)
print("mean-females:", mean_females_int)
print("mean-both:", mean_both_int)


mean-males: 38
mean-females: 39
mean-both: 38


In [170]:
# Total population in Ireland across all ages and sexes
ireland_totals_all = df[df['Administrative Counties'] == 'Ireland'].groupby('Sex')['VALUE'].sum()

# Extract totals
total_males = int(ireland_totals_all.get('Male', 0))
total_females = int(ireland_totals_all.get('Female', 0))
total_both = int(ireland_totals_all.get('Both sexes', total_males + total_females))

# Print results
print("Total population in Ireland (all ages):")
print("Males:", total_males)
print("Females:", total_females)
print("Both sexes:", total_both)


Total population in Ireland (all ages):
Males: 2544549
Females: 2604590
Both sexes: 5149139


part two

In [171]:
# age variable
age = 35

# Define age range (±5 years)
lower_age = age - 5
upper_age = age + 5

In [172]:
# List all regions (Administrative Counties) in the full dataset
regions = df['Administrative Counties'].unique().tolist()
print("List of all regions in Ireland:")
print(regions)


List of all regions in Ireland:
['Ireland', 'Carlow County Council', 'Dublin City Council', 'Dún Laoghaire Rathdown County Council', 'Fingal County Council', 'South Dublin County Council', 'Kildare County Council', 'Kilkenny County Council', 'Laois County Council', 'Longford County Council', 'Louth County Council', 'Meath County Council', 'Offaly County Council', 'Westmeath County Council', 'Wexford County Council', 'Wicklow County Council', 'Clare County Council', 'Cork City Council', 'Cork County Council', 'Kerry County Council', 'Limerick City & County Council', 'Tipperary County Council', 'Waterford City & County Council', 'Galway City Council', 'Galway County Council', 'Leitrim County Council', 'Mayo  County Council', 'Roscommon County Council', 'Sligo County Council', 'Cavan County Council', 'Donegal County Council', 'Monaghan County Council']


In [173]:
# Filter the dataframe for ages within this range
age_group = df[(df['Single Year of Age'] >= lower_age) & (df['Single Year of Age'] <= upper_age)]

# Group by region and sex within the age range, and sum the population
region_totals = ( age_group.groupby(['Administrative Counties', 'Sex'])['VALUE'].sum().unstack(fill_value=0)
)

# Calculate the population difference (Male - Female) for each region
region_totals['Difference'] = region_totals['Male'] - region_totals['Female']

# Find the region with the largest absolute difference
max_region = region_totals['Difference'].abs().idxmax()
max_difference = int(region_totals.loc[max_region, 'Difference'])

# Print results
print("Region with the largest population difference between the sexes in ages", lower_age, "to", upper_age)
print("Region:", max_region)
print("Population difference (Male - Female):", max_difference)


Region with the largest population difference between the sexes in ages 30 to 40
Region: Ireland
Population difference (Male - Female): -30476


In [174]:



# Total population in Ireland for the selected age range
ireland_age_group = age_group[age_group['Administrative Counties'] == 'Ireland']
ireland_age_totals = ireland_age_group.groupby('Sex')['VALUE'].sum()

print("Population in Ireland for ages", lower_age, "to", upper_age)
print(ireland_age_totals)

# Population difference (Male - Female)
ireland_difference = int(ireland_age_totals.get('Male', 0) - ireland_age_totals.get('Female', 0))
print("Population difference (Male - Female):", ireland_difference)


Population in Ireland for ages 30 to 40
Sex
Both sexes    798536
Female        414506
Male          384030
Name: VALUE, dtype: int64
Population difference (Male - Female): -30476


Part 3 : # Find which region has the biggest population difference between the sexes in the selected age group


In [175]:
# Total population in Dublin City Council for the selected age range
dublin_age_group = age_group[age_group['Administrative Counties'] == 'Dublin City Council']
dublin_age_totals = dublin_age_group.groupby('Sex')['VALUE'].sum()

print("Population in Dublin City Council for ages", lower_age, "to", upper_age)
print(dublin_age_totals)

# Population difference (Male - Female)
dublin_difference = int(dublin_age_totals.get('Male', 0) - dublin_age_totals.get('Female', 0))
print("Population difference (Male - Female):", dublin_difference)


Population in Dublin City Council for ages 30 to 40
Sex
Both sexes    120698
Female         59831
Male           60867
Name: VALUE, dtype: int64
Population difference (Male - Female): 1036
